In [ ]:
"""Ignore this code cell, this is an alternate variation I was messing around with at first, start below"""
import requests
import base64

# Replace the values below with your own API key and API secret key
api_key = ""
api_secret_key = ""

# Encode the API key and API secret key in base64
encoded_api_key_secret = base64.b64encode(f"{api_key}:{api_secret_key}".encode()).decode()

# Obtain a Bearer token
headers = {
    "Authorization": f"Basic {encoded_api_key_secret}",
    "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
}
data = {
    "grant_type": "client_credentials"
}
response = requests.post("https://api.twitter.com/oauth2/token", headers=headers, data=data)

# Extract the Bearer token from the response
bearer_token = response.json()["access_token"]

# Use the Bearer token to authenticate requests to the Twitter API
headers = {
    "Authorization": f"Bearer {bearer_token}"
}

# Retrieve the most recent 100 tweets containing the word "NFT"
response = requests.get("https://api.twitter.com/2/tweets/search/recent", headers=headers, params={
    "query": "NFT",
    "max_results": 100
})

# Extract the tweets from the response
tweets = response.json()["data"]

# Print the text of each tweet
#for tweet in tweets:
 #   print(tweet["text"])

df = pd.DataFrame(tweets)

print(df)

In [ ]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = ""


def create_url():
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=created_at,description"
    # You can replace the ID given with the Tweet ID you wish to like.
    # You can find an ID by using the Tweet lookup endpoint
    id = "1602742534366601218"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets/{}/liking_users".format(id)
    return url, user_fields


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2LikingUsersPython"
    return r


def connect_to_endpoint(url, user_fields):
    response = requests.request("GET", url, auth=bearer_oauth, params=user_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url, tweet_fields = create_url()
    json_response = connect_to_endpoint(url, tweet_fields)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()


In [ ]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:en"
start_list = ["2023-01-15T17:00:00Z", "2023-01-16T17:00:00Z", "2023-01-17T17:00:00Z"]
end_list = ["2023-01-15T18:00:00Z", "2023-01-16T18:00:00Z", "2023-01-17T18:00:00Z"]
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                write_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                write_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

In [ ]:
import csv
import time


def tweets_per_hour_range(keyword: str):
    """This function returns 10,000 tweets (containing keyword) per 1 hour range from 100 random hour ranges in the past year."""
    bearer_token = auth()
    headers = create_headers(bearer_token)
    # Creates 100 random one hour ranges in the past year
    
    start_list = list()
    end_list = list()
    for _ in range(0, 100):
        start_time, end_time = random_date()
        while start_time in start_list:
            start_time, end_time = random_date()
        start_list.append(start_time)
        end_list.append(end_time)
    
    # Counts total tweets, not tweets per time period
    total_tweets: int = 0
    # Max results per enpoint call, system limit is 100
    max_results: int = 100 

    # Create file data.csv
    csvFile = open("data.csv", "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['author id', 'created_at', 'id', 'like_count', 'reply_count','retweet_count', 'source', 'tweet'])
    csvFile.close()

    for i in range(0, len(start_list)):

        # Inputs
        count: int = 0 # Counts tweets per time period
        max_count: int = 1 # Input for max tweets per time period
        # Total tweets will be max_count times the amount of time ranges
        flag: bool = True
        next_token: str = None
    
        # Check if flag is true
        while flag:
            # Check if max_count reached
            if count >= max_count:
                break
            print("-------------------")
            print("Token: ", next_token)
            url = create_url(keyword, start_list[i],end_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            result_count = json_response['meta']['result_count']

            if 'next_token' in json_response['meta']:
                # Save the token to use for next call
                next_token = json_response['meta']['next_token']
                print("Next Token: ", next_token)
                if result_count is not None and result_count > 0 and next_token is not None:
                    #write_to_csv('data.json', 'data.csv')
                    write_to_csv(json_response, 'data.csv')
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)                
            # If no next token exists
            else:
                if result_count is not None and result_count > 0:
                    print("-------------------")
                    print("Start Date: ", start_list[i])
                    #write_to_csv('data.json', 'data.csv')
                    write_to_csv(json_response, 'data.csv')
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)
            
                #Since this is the final request, turn flag to false to move to the next time period.
                flag = False
                next_token = None
            time.sleep(5)
    print("Total number of results: ", total_tweets)

In [ ]:
import dateutil
'''
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 
'''

In [ ]:
"""WORKING EXAMPLE OF SO FAR, Above is for more function use etc."""
import json
import pandas as pd
import csv

bearer_token = auth()
"""input the necessary inputs below."""
headers = create_headers(bearer_token)
keyword = "Smartchain OR Airdrop OR Crypto"
# Use the commented out code below when using full access api
#start_time, end_time = random_date()
start_time = "2023-01-02T17:00:00Z"
end_time = "2023-01-04T20:00:00Z"

max_results = 100
url = create_url(keyword, start_time, end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
result_dict = json_response["data"]
#while "next_token" in json_response["meta"] and len(result_dict) < 9900:
    #print(json.dumps(json_response, indent=4, sort_keys=True))
    #json_response = connect_to_endpoint(url[0], headers, url[1], json_response["meta"]["next_token"])
    #result_dict.extend(json_response["data"])
#print(json.dumps(json_response, indent=4, sort_keys=True))
print(json.dumps(result_dict, indent=4, sort_keys=True))

#def append_to_csv(json_response, fileName):
with open('data.json', 'w') as f:
    json.dump(json_response, f)
#df = pd.read_json('data.json')
write_to_csv()
#url = create_url(keyword, start_time, end_time, max_results)
#json_response = connect_to_endpoint(url[0], headers, url[1])
#print(json.dumps(json_response, indent=4, sort_keys=True))